<a href="https://colab.research.google.com/github/kristynpantoja/math689project/blob/master/AVITMtoLDAworkingversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math

In [0]:
class ProdLDA(nn.Module):

    def __init__(self, net_arch):
        super(ProdLDA, self).__init__()
        ac = net_arch
        self.net_arch = net_arch
        # encoder
        self.en1_fc     = nn.Linear(ac.num_input, ac.en1_units)             # 1995 -> 100
        self.en2_fc     = nn.Linear(ac.en1_units, ac.en2_units)             # 100  -> 100
        self.en2_drop   = nn.Dropout(0.2)
        self.mean_fc    = nn.Linear(ac.en2_units, ac.num_topic)             # 100  -> 50
        self.mean_bn    = nn.BatchNorm1d(ac.num_topic)                      # bn for mean
        self.logvar_fc  = nn.Linear(ac.en2_units, ac.num_topic)             # 100  -> 50
        self.logvar_bn  = nn.BatchNorm1d(ac.num_topic)                      # bn for logvar
        # z
        self.p_drop     = nn.Dropout(0.2)
        # decoder
        self.decoder = nn.Linear(ac.num_topic, ac.num_input)      # 50   -> 1995
        self.decoder_bn = nn.BatchNorm1d(ac.num_topic)                      # bn for decoder
        # prior mean and variance as constant buffers
        prior_mean   = torch.Tensor(1, ac.num_topic).fill_(0)
        prior_var    = torch.Tensor(1, ac.num_topic).fill_(ac.variance)
        prior_logvar = prior_var.log()
        self.register_buffer('prior_mean',    prior_mean)
        self.register_buffer('prior_var',     prior_var)
        self.register_buffer('prior_logvar',  prior_logvar)
        # initialize decoder weight
        if ac.init_mult != 0:
            #std = 1. / math.sqrt( ac.init_mult * (ac.num_topic + ac.num_input))
            self.decoder.weight.data.uniform_(0, ac.init_mult)
        # remove BN's scale parameters
#         self.logvar_bn .register_parameter('weight', None)
#         self.mean_bn   .register_parameter('weight', None)
#         self.decoder_bn.register_parameter('weight', None)
#         self.decoder_bn.register_parameter('weight', None)
        self.beta = nn.Parameter(torch.randn([self.net_arch.num_input, self.net_arch.num_topic]))

    def forward(self, input, compute_loss=False, avg_loss=True):
        # compute posterior
        assert input.shape[1] == doc_term_matrix_tensor.shape[1], "input isn't batch size x vocab size"
        en1 = F.softplus(self.en1_fc(input))                            # en1_fc   output
        en2 = F.softplus(self.en2_fc(en1))                              # encoder2 output
        en2 = self.en2_drop(en2)
        posterior_mean   = self.mean_bn  (self.mean_fc  (en2))          # posterior mean
        posterior_logvar = self.logvar_bn(self.logvar_fc(en2))          # posterior log variance
        posterior_var    = posterior_logvar.exp()
        # take sample
        eps = Variable(input.data.new().resize_as_(posterior_mean.data).normal_()) # noise
        z = posterior_mean + posterior_var.sqrt() * eps                 # reparameterization
        assert z.shape[1] == self.net_arch.num_topic, "hidden variable z (from TR) isn't batch size x num_topic"
        # get theta
        p = F.softmax(z)                                                # mixture probability
        p = self.p_drop(p)
        assert p.shape[1] == self.net_arch.num_topic, "p (theta) isn't same size as z"
        # return beta times theta, i.e. VxK times batch_size x K, need to transpose theta first
        # do reconstruction
#         print(self.beta.sum())
        recon = F.softmax(self.decoder_bn(self.beta), dim=0).mm(p.t()).t()          # reconstructed distribution over vocabulary
#         recon = p.mm(F.softmax(self.decoder_bn(self.beta), dim=0).t())
        assert input.shape[1] == doc_term_matrix_tensor.shape[1], "output isn't batch size x vocab size"
        
        if compute_loss:
            return recon, self.loss(input, recon, posterior_mean, posterior_logvar, posterior_var, avg_loss)
        else:
            return recon

    def loss(self, input, recon, posterior_mean, posterior_logvar, posterior_var, avg=True):
        # NL
        NL  = -(input * (recon+1e-10).log()).sum(1) # vector with batch-size number of elements
        # KLD, see Section 3.3 of Akash Srivastava and Charles Sutton, 2017, 
        # https://arxiv.org/pdf/1703.01488.pdf
        prior_mean   = Variable(self.prior_mean).expand_as(posterior_mean) # batch-size x num_topics
        prior_var    = Variable(self.prior_var).expand_as(posterior_mean)
        prior_logvar = Variable(self.prior_logvar).expand_as(posterior_mean)
        var_division    = posterior_var  / prior_var
        diff            = posterior_mean - prior_mean
        diff_term       = diff * diff / prior_var
        logvar_division = prior_logvar - posterior_logvar
        # put KLD together
        KLD = 0.5 * ( (var_division + diff_term + logvar_division).sum(1) - self.net_arch.num_topic )
#         print(KLD.mean())
        # loss
        loss = (NL + KLD)
        # in traiming mode, return averaged loss. In testing mode, return individual loss
        if avg:
            return loss.mean() # averaged over all the documents in the batch (1/batch_size)*sum
        else:
            return loss
          
def train():
    for epoch in range(args.num_epoch):
        all_indices = torch.randperm(doc_term_matrix_tensor.size(0)).split(args.batch_size)
        loss_epoch = 0.0
        model.train()                   # switch to training mode
        for batch_indices in all_indices:
            if not args.nogpu: batch_indices = batch_indices.cuda()
            input = Variable(doc_term_matrix_tensor[batch_indices])
#             print(batch_indices.shape)
#             print(input.shape)
            recon, loss = model(input, compute_loss=True)
            # optimize
            optimizer.zero_grad()       # clear previous gradients
            loss.backward()             # backprop
            optimizer.step()            # update parameters
            # report
            loss_epoch += loss.data[0]    # add loss to loss_epoch, then take the average in the print statement
        if epoch % 5 == 0:
            print('Epoch {}, loss={}'.format(epoch, loss_epoch / len(all_indices)))

In [0]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
# categories = ['talk.politics.guns', 'sci.space', 'soc.religion.christian',
#               'misc.forsale', 'rec.sport.baseball', 'comp.sys.mac.hardware']
categories = ['talk.politics.guns', 'sci.space']
# newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_train = fetch_20newsgroups(subset='train')


In [0]:
vectorizer = CountVectorizer(stop_words = 'english', min_df=.01, max_df=0.9, 
                             token_pattern = u'(?ui)\\b[a-z]{3,}\\b')
count_vecs = vectorizer.fit_transform(newsgroups_train.data)
doc_term_matrix = count_vecs.toarray()
doc_term_matrix.shape # number of documents, number of words (in vocab)
tokenizer = vectorizer.build_tokenizer()

In [0]:
doc_term_matrix_tensor = torch.from_numpy(doc_term_matrix).float()

In [0]:
import argparse
from types import SimpleNamespace

args_dict = {"en1_units" : 100, "en2_units" : 100, "num_topic" : 50, 
             "batch_size" : 200, "optimizer" : 80, "learning_rate" : 0.002, 
             "momentum" : 0.99, "num_epoch" : 80, "init_mult" : 1, 
             "variance" : 0.995, "start" : True, "nogpu" : True}
args = SimpleNamespace(**args_dict)
args.num_input = doc_term_matrix_tensor.shape[1]

In [0]:
model = ProdLDA(args)

In [0]:
optimizer = torch.optim.Adam(model.parameters(), args.learning_rate, betas=(args.momentum, 0.999))

In [18]:
train()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:106: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 0, loss=621.9462890625
Epoch 5, loss=584.5128173828125
Epoch 10, loss=566.5389404296875
Epoch 15, loss=560.741455078125
Epoch 20, loss=559.405029296875
Epoch 25, loss=559.4723510742188
Epoch 30, loss=558.0155639648438
Epoch 35, loss=555.9494018554688
Epoch 40, loss=556.5142822265625
Epoch 45, loss=555.7542724609375
Epoch 50, loss=554.3941650390625
Epoch 55, loss=553.204345703125
Epoch 60, loss=552.0199584960938
Epoch 65, loss=550.1524658203125
Epoch 70, loss=551.4306640625
Epoch 75, loss=549.3037719726562


In [0]:
associations = {
    'jesus': ['prophet', 'jesus', 'matthew', 'christ', 'worship', 'church'],
    'comp ': ['floppy', 'windows', 'microsoft', 'monitor', 'workstation', 'macintosh', 
              'printer', 'programmer', 'colormap', 'scsi', 'jpeg', 'compression'],
    'car  ': ['wheel', 'tire'],
    'polit': ['amendment', 'libert', 'regulation', 'president'],
    'crime': ['violent', 'homicide', 'rape'],
    'midea': ['lebanese', 'israel', 'lebanon', 'palest'],
    'sport': ['coach', 'hitter', 'pitch'],
    'gears': ['helmet', 'bike'],
    'nasa ': ['orbit', 'spacecraft'],
}
def identify_topic_in_line(line):
    topics = []
    for topic, keywords in associations.items():
        for word in keywords:
            if word in line:
                topics.append(topic)
                break
    return topics
def print_top_words(beta, feature_names, n_top_words=10):
    print('---------------Printing the Topics------------------')
    for i in range(len(beta)):
        line = " ".join([feature_names[j] 
                            for j in beta[i].argsort()[:-n_top_words - 1:-1]])
        topics = identify_topic_in_line(line)
        print('|'.join(topics))
        print('     {}'.format(line))
    print('---------------End of Topics------------------')

In [20]:
emb = model.decoder.weight.data.cpu().numpy().T
print("shape of beta is " + str(emb.shape))
print_top_words(emb, vectorizer.get_feature_names())

shape of beta is (50, 1739)
---------------Printing the Topics------------------
car  
     engineering entirely father happened remove supposed guys discussion previous needed

     really type men peter claims version machine court greek time

     carry fine sport brad years misc straight define groups federal

     didn faster apart edward guess leave driving network link aren

     copies style today questions add phone clock solution problem men

     curious notes traffic adams playoffs risk does public straight sport

     march sending thanks field standard keeping dealer just include excuse

     ucs shown wondering kent bought messages example gone believed committed

     clear ask available daniel citizens april alive different vga advance

     sex rights sample electronic larry absolute security social source heard

     heard controller love society tell mil auto sure held believe

     huge attention print comes define eat surface keyboard star happen

     wings revie

In [24]:
emb = model.beta.detach().numpy().T
print("shape of beta is " + str(emb.shape))
print_top_words(emb, vectorizer.get_feature_names())

shape of beta is (50, 1739)
---------------Printing the Topics------------------

     edu com university like nntp host does year reply really
jesus
     max com jesus article son matthew new reply word men

     just like don writes people good time article does think

     com article university world use like need make distribution really

     writes make use want time said right government new need

     edu available use file program information ftp image pub data
jesus
     god jesus does believe people say think true christian question

     writes com know does like just good time people new

     don host like just use time need things good com

     just com like article new think know does say good
polit
     people said know going armenian time government years president armenia

     com writes article posting don nntp like know distribution new

     edu article writes university posting nntp just host like distribution

     edu com article writes posting nntp host uni

In [0]:
beta = model.decoder.weight.detach()#.softmax(0)
print(beta.sum(0))
print(beta.shape)
_, ind = torch.sort(beta, 0)
print(ind.shape)
# ind.numpy()[0:50, 0] - ind.numpy()[0:50, 1]
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 0])
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 1])
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 2])
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 3])
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 4])
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 5])
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 6])
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 7])
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 8])
print(np.array(sorted(vectorizer.get_feature_names(), key = lambda x: x[1]))[ind.numpy()][0:25, 9])

# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:25, 4])

In [0]:
print(model.decoder.weight)
model.decoder.weight.data.cpu()

In [0]:
from itertools import product

def topic_coherence(beta, M, doc_term_matrix):
  K = beta.shape[1] # beta has dim V x K
  coherences = np.zeros(K)
  for t in range(K):
    index = np.argsort(-beta[:, t])[0:M]
    cart_prod = product(list(index), list(index))
    for ind1, ind2 in cart_prod:
      if ind1 == ind2:
        pass
      else:
        d_ind1 = (doc_term_matrix[:, ind1] > 0).sum()
        d_ind12 = ((doc_term_matrix[:, ind1] > 0) & (doc_term_matrix[:, ind2] > 0)).sum()
        coherences[t] += np.log1p(d_ind12) - np.log(d_ind1)

  return coherences

In [25]:
topic_coherence(model.beta.detach().numpy(), 20, doc_term_matrix)

array([-650.88923448, -817.20633464, -429.45869701, -522.86269983,
       -556.02409611, -770.52725375, -578.23299271, -430.73181983,
       -517.84426803, -515.44595657, -648.47097528, -478.74120291,
       -500.46485713, -471.79991538, -500.91968408, -601.84733158,
       -421.96658315, -419.21672536, -465.58039039, -761.66384369,
       -510.87669811, -429.1398398 , -593.91115552, -502.82021129,
       -565.29847244, -451.25286429, -588.95802632, -504.10260176,
       -755.20671554, -533.82540282, -615.30444371, -437.38115235,
       -518.48944887, -436.40228551, -531.05632365, -557.23560556,
       -471.8931654 , -553.10739077, -484.81458719, -568.95545001,
       -539.2576399 , -736.20418651, -636.66219113, -469.21569111,
       -473.07713594, -773.87512275, -588.82486116, -639.52316987,
       -503.64257899, -473.20491048])

In [0]:
sorted(vectorizer.vocabulary_, key = lambda x: x[1])[0]

In [0]:
vocab = {"hi": 13, "bye": 2, "hello": 3}
foo = zip(*sorted(vocab.items(), key = lambda x: x[1]))
list(foo)

In [0]:
def print_top_words(beta, feature_names, n_top_words=10):
    print('---------------Printing the Topics------------------')
    for i in range(len(beta)): # for all the rows (words in vocab) in beta,
        line = " ".join([feature_names[j] 
                            for j in beta[i].argsort()[:-n_top_words - 1:-1]])
#         topics = identify_topic_in_line(line)
#         print('|'.join(topics))
        print('     {}'.format(line))
    print('---------------End of Topics------------------')

In [0]:
print_top_words(beta.numpy().T, sorted(vectorizer.vocabulary_, key = lambda x: x[1]))

In [0]:
np.bincount(np.array([4,6,3,6,8,2,6,78,89,5]))

In [0]:
"?".join("bsc")

In [0]:
doc_term_matrix.